# Import the library

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# define the function to select region of interest 

In [2]:

def filter_region(image, vertices):
    """
    Create the mask using the vertices and apply it to the input image
    """
    mask = np.zeros_like(image)
    if len(mask.shape)==2:
        cv2.fillPoly(mask, vertices, 255)
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2]) # in case, the input image has a channel dimension        
    return cv2.bitwise_and(image, mask)

In [3]:

def select_region(image):
    """
    It keeps the region surrounded by the `vertices` (i.e. polygon).  Other area is set to 0 (black).
    """
    # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.05, rows*0.77]
    top_left     = [cols*0.3, rows*0.55]
    bottom_right = [cols*0.90, rows*0.77]
    top_right    = [cols*0.6, rows*0.55] 
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return filter_region(image, vertices)
    

# Capture a Video 
apply the edge detection and apply the HoughTransform on the top of the image.
call the ROI function

In [4]:
video = cv2.VideoCapture("High speed.mp4")
while True:
    ret, frame = video.read()
    if not ret:
        video = cv2.VideoCapture("High speed.mp4")
        continue
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #low_yellow = np.array([18,94,140])
    #up_yellow = np.array([48,255,255])
    #mask = cv2.inRange(hsv,low_yellow,up_yellow)
    lower_white = np.array([0,0,0], dtype=np.uint8)
    upper_white = np.array([0,0,255], dtype=np.uint8)
   # mask = cv2.inRange(hsv,lower_white,upper_white)
    
   
    edges = cv2.Canny(frame,75,150)
    image = select_region(edges)
    lines = cv2.HoughLinesP(image,1,np.pi/180,50,maxLineGap =50)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),2)
   
    cv2.imshow("frame",frame)
    
    
    key = cv2.waitKey(25)
    if key == 27:
        cv2.destroyAllWindows()
        break
video.release()